<a href="https://colab.research.google.com/github/YagmurTaze/OCR-ICR/blob/main/1_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocess

In [5]:
from tqdm import tqdm

# Verilerinizi yükleyin
images = []
labels = []

path = '/content/drive/MyDrive/Datasets/TURKISH CHARACTERS'

dir_list = os.listdir(path)
for i in dir_list:
    dir = os.path.join(path, i)
    file_list = os.listdir(dir)
    for j in file_list:
        files = os.path.join(dir, j)
        img = cv2.imread(files)
        img = cv2.resize(img, (64, 64))
        img = np.array(img, dtype=np.float32)
        img = img / 255
        images.append(img)
        labels.append(i)

In [6]:
X = np.array(images)
y = np.array(labels)

## The Model

In [7]:
X = np.array(images)
y = np.array(labels)

# Etiketleri kodlayın
le = LabelEncoder()
y = le.fit_transform(y)

# Veriyi eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli oluşturun
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=68, activation='softmax'))

In [8]:
# Modeli derleyin
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitin
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=5)

# Modeli kaydedin
model.save("byLettermodel.h5")


Epoch 1/5
340/340 [==============================] - 27s 74ms/step - loss: 3.1132 - accuracy: 0.2131 - val_loss: 1.8150 - val_accuracy: 0.4853
Epoch 2/5
340/340 [==============================] - 25s 75ms/step - loss: 1.3158 - accuracy: 0.6026 - val_loss: 1.1006 - val_accuracy: 0.6691
Epoch 3/5
340/340 [==============================] - 25s 73ms/step - loss: 0.8185 - accuracy: 0.7200 - val_loss: 0.9573 - val_accuracy: 0.6978
Epoch 4/5
340/340 [==============================] - 26s 77ms/step - loss: 0.6212 - accuracy: 0.7757 - val_loss: 0.8571 - val_accuracy: 0.7213
Epoch 5/5
340/340 [==============================] - 25s 73ms/step - loss: 0.4892 - accuracy: 0.8143 - val_loss: 0.8329 - val_accuracy: 0.7301


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Testing

In [9]:
from tensorflow.keras.models import load_model

model = load_model("byLettermodel.h5")

In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Test edilecek resmi yükleyin
test_image = image.load_img('/content/C1.png', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Tahmin yapma
result = model.predict(test_image)

predicted_class_index = np.argmax(result)
alphabet = '0123456789ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZabcçdefgğhıijklmnoöprsştuüvyz'
predicted_letter = alphabet[predicted_class_index]

print("Tahmin edilen harf:", predicted_letter)

1/1 [==============================] - 0s 116ms/step
Tahmin edilen harf: C
